Below is the code and output from my first homework assignment from my Algorithms for Data Science course from my Master's in Data Science program from Columbia University (from the Fall semester of 2020). Please note that this notebook is not maintained, so it may not run correctly for someone else.

# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement depth-first search for identifying the connected components of an undirected graph, implementing procedure Search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [1]:
# USE NETWORKX FOR GRAPHS, PULL IN DATA
import networkx
homer = open('homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

Next implement a function to read in the edges from the input file.

In [2]:
def read_nodes(gfile):
    """Reads in the nodes of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top.
        
    Returns:
        A list of the nodes in the graph of the form:
            ['CH', 'AG, 'ME', ...]
    """
    
    # MAKE EMPTY LIST TO APPEND TO
    nodes = []
    

    
    # NEXT I WANT TO PULL THE FIRST TWO LETTERS FROM EACH LINE, 
    # EXCLUDING THOSE THAT BEGIN WITH *, &:, OR A NUMERIC THEN :
    # INITIAL DATA EXPLORATION SHOWED THAT, CONVENIENTLY, CHAPTER LINES (I.E. THOSE BEGINNING WITH :) 
    # DON'T CONTAIN SPACES; THIS MEANS THAT ONCE SPLIT THEY HAVE LENGTHS OF 1, 
    # SO I WILL IGNORE OBJECTS OF LENGTH ONE OR THOSE WHOSE FIRST ITEM IS *
    # ALSO USE SEEK SO THAT THE FILE CAN BE READ MULTIPLE TIMES
    
    gfile.seek(0)
    for line in gfile:
        split_item = line.split()
        if len(split_item) > 1 and split_item[0] != '*':
            nodes.append(split_item[0])
    return nodes

    pass

In [3]:
def read_edges(gfile):
    """Reads in the edges of the graph from the input file.
    
    Args:
        gfile: A handle for the file containing the graph data, starting at the top 
            of the edges section.
            
    Returns:
        The edges in the graph as a list of pairs of the form:
            [('CH', 'AG'), ('AG', 'ME'), ...]
    """
    # TODO: implement function
    
    # RESET TO BEGINNING OF FILE AND MAKE EMPTY LISTS
    
    gfile.seek(0)
    larry = gfile

    groups = []
    edges = []

    # NEXT I ITERATE OVER EACH LINE, STRIPPING \n AND SPLITTING AT SEMI-COLONS IN ORDER TO GET GROUPS ISOLATED
    # IF A LINE DOESN'T CONTAIN A SEMI-COLON (I.E. IT IS OF LENGTH 0 AFTER SPLITTING) I ASSUME IT IS NOT RELEVANT HERE
    # FOR THE FIRST GROUP IN EACH LINE, I USE ANOTHER SPLIT AT ":" TO GET RID OF CHAPTER NOTATIONS
    # I APPEND EACH GROUP TO MY GROUPS LIST
    for line in larry:
        line = line.strip()
        split_item = line.split(';')
        if len(split_item) > 1:
            for i in range(len(split_item)):
                if i == 0:
                    groups.append(split_item[i].split(":")[1].split(','))
                else:
                    groups.append(split_item[i].split(','))

    # WITHIN EACH GROUP I TAKE EACH CHARACTER AND MAKE A PAIRING WITH EACH CHARACTER ALSO IN THE GROUP
    # I ONLY PAIR CHARACTERS WITH THOSE COMING AFTER IN ORDER TO AVOID DUPLICATES (UNDIRECTED GRAPH)
    for group in groups:
        for i in range(len(group)):
            for j in range(i+1, len(group)):
                if group[i] != group[j]:
                    edge_larry = (group[i], group[j])
                    edges.append(edge_larry)
    return edges
    pass

The following code should now correctly create the graph.

In [4]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement procedure Search. The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [5]:
def Search(graph, root):
    """Runs Search from vertex root in a graph. Neighboring nodes are processed in alphabetical order.
    
    Args:
        graph: the given graph, with nodes encoded as strings.
        root: the node from which to start the search.
        
    Returns:
        A list of nodes in the order in which they were first visited.
    """
    # COPYING GRAPH
    
    graph = graph.copy()
    
    # DEFINING DFS-VISIT
    
    def dfs_visit(g, u, time):
        mapped = [] #INITIALIZED W/NO NODES MAPPED
        
        time += 1 #INCREMENT TIME BY 1 EACH TIME SEARCH IS RUN
        g.nodes[u]['start'] = time #ROOT REPS NEXT NODE CONSIDERED, SO IT'S START IS "NOW"
        g.nodes[u]['color'] = 'gray'
        mapped.append(u) #ADD TO MAPPED
        
        for v in sorted(g.neighbors(u)):
            if g.nodes[v]['color'] == 'white':
                g.nodes[v]['predecessor'] = g.nodes[u] #DEFINE ROOT AS PREDECESSOR FOR ALL NEIGHBORS
                
                #EXPAND MAPPED W/ THE NEIGHBOR
                mapped.extend(dfs_visit(g, v, time)) 
                #NOTE THAT THIS SHOULD CYCLE THRU V'S NEIGHBORS AND NEIGHBORS' NEIGHBORS AND SO FORTH THRU THE GRAPH
                
        g.nodes[u]['color'] = 'black'    
        time += 1
        g.nodes[u]['finish'] = time
        
        return mapped
        
    # THE MAIN DFS FUNCTION, CALLING THE NOW-DEFINED DFS-VISIT FUNCTION
    for u in graph.nodes():
        graph.nodes[u]['color'] = 'white'
        graph.nodes[u]['predecessor'] = None
        
    time = 0
    
    for u in graph.nodes():
        if graph.nodes[u]['color'] == 'white':
            return dfs_visit(graph, root, time)
                
    pass   

We will check the correctness of your code by verifying that it correctly computes the connected component of node Ulysses (node `OD`).

In [6]:
ulysses = Search(G, 'OD')

Next implement DFS to find the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your Search routine, this will ensure that the output is again uniquely defined.

In [7]:
def connected_components(graph):
    """Computes the connected components of the given graph.
    
    Args: 
        graph: the given graph, with nodes encoded as strings.
        
    Returns:
        The connected components of the graph. Components are listed in
        alphabetical order of their root nodes.
    """
    # TODO: implement function
    
    # MAKING EMPTY OF ALL CCs TO EVENTUALLY RETURN 
    full_list_of_cc = []
    
    # MAKING WHILE LOOP TO RUN SEARCH ON SMALLEST COMPONENT LEFT IN LIST 
    # THEN ADD TO FULL CC LIST AND TOSS THAT NODE OUT OF ALL_NODES 
    # UNTIL ALL NODES HAVE BEEN DISCOVERED
    
    while len(graph.nodes()) > 0 :
        all_nodes = sorted(graph.nodes())
        cc = Search(graph, all_nodes[0])
        full_list_of_cc.append(cc)
        graph.remove_nodes_from(cc)
    return full_list_of_cc
    pass

We will check correctness of your code by verifying that your output is identical to our solution.

In [8]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [9]:
component_sizes = [len(c) for c in character_interactions]
print("There are 12 connected components in the Iliad:{}".format(len(component_sizes) == 12))
print("The giant component has size 542:{}".format(max(component_sizes) == 542))
print("There are 5 isolated characters:{}".format(len([c for c in component_sizes if c == 1]) == 5))

There are 12 connected components in the Iliad:True
The giant component has size 542:True
There are 5 isolated characters:True
